In [1]:
import pandas as pd

#url = "https://github.com/AquaticEcoDynamics/tassielakes-data/raw/main/data-lake/HT/Hydrology/454.1_LakeRiverBLWoodsLakeDam_ChannelFlow_Continuous.csv"

# Read the CSV file and select only the first column
df = pd.read_csv("../../data-lake/HT/Hydrology/454.1_LakeRiverBLWoodsLakeDam_ChannelFlow_Continuous.csv", usecols=[0], header=None, encoding='unicode_escape')
#df = pd.read_csv(url, usecols=[0], header=None, encoding='unicode_escape')

# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Name the first column
df.columns = ["Raw"]

# Split the 'Raw' column by multiple spaces
split_data = df["Raw"].str.split(r'\s+', expand=True)
#print(split_data)

# Split the 'Raw' column by multiple spaces
split_data = df["Raw"].str.split(r'\s+', expand=True)

# Extract Date and Time
df['Date'] = split_data[0] + ' ' + split_data[2]

# Extract Data value
df['Data'] = split_data[3]

# Drop the original 'Raw' column
df = df[['Date', 'Data']]

# Convert 'Data' to numeric
df['Data'] = pd.to_numeric(df['Data'], errors='coerce')
print(df)


                        Date      Data
0        11/03/1999 13:00:00  0.000000
1        12/03/1999 02:45:00  0.000000
2        12/03/1999 03:00:00  0.000000
3        12/03/1999 11:00:00  0.000000
4        12/03/1999 11:15:00  0.000000
...                      ...       ...
1263124  16/04/2024 09:00:00  0.380967
1263125  16/04/2024 09:15:00  0.380967
1263126  16/04/2024 09:30:00  0.380967
1263127  16/04/2024 09:45:00  0.380967
1263128  16/04/2024 10:00:00  0.380967

[1263129 rows x 2 columns]


In [2]:
# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# Drop rows with NaT in 'Date' column
df = df.dropna(subset=['Date'])

print(df)

                       Date      Data
0       1999-03-11 13:00:00  0.000000
1       1999-03-12 02:45:00  0.000000
2       1999-03-12 03:00:00  0.000000
3       1999-03-12 11:00:00  0.000000
4       1999-03-12 11:15:00  0.000000
...                     ...       ...
1263124 2024-04-16 09:00:00  0.380967
1263125 2024-04-16 09:15:00  0.380967
1263126 2024-04-16 09:30:00  0.380967
1263127 2024-04-16 09:45:00  0.380967
1263128 2024-04-16 10:00:00  0.380967

[1263129 rows x 2 columns]


In [3]:
# Group by 'Date' and average the 'Data' values for duplicate timestamps
df = df.groupby('Date').mean().reset_index()

# Set 'Date' as index for resampling
df.set_index('Date', inplace=True)

# Resample the DataFrame to 15-minute intervals and interpolate
df = df.resample('15T').interpolate(method='linear')

# Reset the index to convert it back to a column
df = df.reset_index()

# Resample the DataFrame to hourly intervals and interpolate
# df = df.resample('H').interpolate(method='linear')

# Format the 'Date' column
df['Date'] = df['Date'].dt.strftime('%d/%m/%Y %H:%M:%S')

print(df)

                       Date      Data
0       11/03/1999 13:00:00  0.000000
1       11/03/1999 13:15:00  0.000000
2       11/03/1999 13:30:00  0.000000
3       11/03/1999 13:45:00  0.000000
4       11/03/1999 14:00:00  0.000000
...                     ...       ...
880112  16/04/2024 09:00:00  0.380967
880113  16/04/2024 09:15:00  0.380967
880114  16/04/2024 09:30:00  0.380967
880115  16/04/2024 09:45:00  0.380967
880116  16/04/2024 10:00:00  0.380967

[880117 rows x 2 columns]


In [4]:
# Write the DataFrame to a CSV file
df.to_csv("outflow_woods_dam-spillway_19990311-20240416.csv", index=False)